In [3]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = HuggingFacePipeline.from_model_id(
    model_id="openai-community/gpt2",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 150,
    },
)

chain = prompt | llm

print(chain.invoke({"word": "tomato"}))

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 common seed. It contains up to 40% sugar. There are many varieties so I always put them in their own bottle or in one I've left over from when I used it to fill up. While most people do this on their tomatoes, there are many to choose from; here are my favorites.

Ingredients:

1 large tomato, cut into about ¼-inch pieces

2 cups chopped walnuts, cut into ¼-inch pieces

1 1/2 cups (800 grams) granulated sugar

1 package (400 grams) nutritional yeast

1 teaspoon baking powder

4 tablespoons chopped black pepper

Cream your canned tomatoes and walnuts together with ¼-inch each. Add
